# Introduction

In [1]:
%ls

algorithms/    Faster2.ipynb  measuring/          profiling_presentation.pdf
bin/           faster.zip     pi/                 share/
caching/       include/       pip-selfcheck.json
Faster1.ipynb  lib/           profiling.pdf


In [2]:
%less Faster1.ipynb

In [3]:
cd measuring/

/home/mark/Projects/jupyter/pycon2017t1/measuring


In [4]:
%ls

accumulate.py             memory_size_hpy.py
accumulate.py.lprof       memory_size_pympler.py
calc.py                   profile_me.py
calc.py.lprof             profile_me.py.prof
clock_check.py            profile_me_use_line_profiler.py
cpu_time.py               profile_me_use_line_profiler.py.lprof
create_list.py            profile_speed.py
fast.stats                pympler_list_growth.py
fast.txt                  pystone2_3.py
list_alloc_steps.py       pystone2.py
local_ref.py              pystone3.py
make_list_gen.py          pystone_converter.py
memory_growth_hpy.py      use_mem.py
memory_growth_pympler.py


# Profiling CPU

`%load profile_me.py` and run it

In [5]:
# %load profile_me.py
# file profile_me.py

"""Example to be profiled.
"""
import sys
import time

if sys.version_info.major < 3:
    range = xrange


def fast():
    """Wait 0.001 seconds.
    """
    time.sleep(1e-3)


def slow():
    """Wait 0.1 seconds.
    """
    time.sleep(0.1)


def use_fast():
    """Call `fast` 100 times.
    """
    for _ in range(100):
        fast()


def use_slow():
    """Call `slow` 100 times.
    """
    for _ in range(100):
        slow()


if __name__ == '__main__':
    use_fast()
    use_slow()


In [6]:
import cProfile

In [7]:
profile = cProfile.Profile()

In [8]:
profile.runcall(use_fast)

In [9]:
profile.print_stats()

         202 function calls in 0.109 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.001    0.000    0.108    0.001 <ipython-input-5-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.109    0.109 <ipython-input-5-31b8b4df29c5>:25(use_fast)
      100    0.108    0.001    0.108    0.001 {built-in method sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [10]:
profile2 = cProfile.Profile()

In [11]:
profile2.runcall(use_slow)

In [12]:
profile2.print_stats()

         202 function calls in 10.024 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.001    0.000   10.023    0.100 <ipython-input-5-31b8b4df29c5>:19(slow)
        1    0.001    0.001   10.024   10.024 <ipython-input-5-31b8b4df29c5>:32(use_slow)
      100   10.022    0.100   10.022    0.100 {built-in method sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [13]:
cProfile.run('use_fast()')

         204 function calls in 0.119 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.000    0.000    0.119    0.001 <ipython-input-5-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.119    0.119 <ipython-input-5-31b8b4df29c5>:25(use_fast)
        1    0.000    0.000    0.119    0.119 <string>:1(<module>)
        1    0.000    0.000    0.119    0.119 {built-in method exec}
      100    0.119    0.001    0.119    0.001 {built-in method sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [14]:
use_fast.__name__

'use_fast'

In [15]:
cProfile.run('use_fast()', 'fast.stats')

In [16]:
ls *.stats

fast.stats


In [17]:
import pstats

In [18]:
stats = pstats.Stats('fast.stats')

In [19]:
stats.print_stats()

Thu May 18 11:31:34 2017    fast.stats

         204 function calls in 0.111 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.111    0.111 {built-in method exec}
        1    0.000    0.000    0.111    0.111 <string>:1(<module>)
        1    0.001    0.001    0.111    0.111 <ipython-input-5-31b8b4df29c5>:25(use_fast)
      100    0.110    0.001    0.110    0.001 {built-in method sleep}
      100    0.001    0.000    0.111    0.001 <ipython-input-5-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [20]:
stats.sort_stats('time').print_stats()

Thu May 18 11:31:34 2017    fast.stats

         204 function calls in 0.111 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.110    0.001    0.110    0.001 {built-in method sleep}
      100    0.001    0.000    0.111    0.001 <ipython-input-5-31b8b4df29c5>:13(fast)
        1    0.001    0.001    0.111    0.111 <ipython-input-5-31b8b4df29c5>:25(use_fast)
        1    0.000    0.000    0.111    0.111 {built-in method exec}
        1    0.000    0.000    0.111    0.111 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [21]:
stats.print_callers('fast')

   Ordered by: internal time
   List reduced from 6 to 2 due to restriction <'fast'>

Function                                     was called by...
                                                 ncalls  tottime  cumtime
<ipython-input-5-31b8b4df29c5>:13(fast)      <-     100    0.001    0.111  <ipython-input-5-31b8b4df29c5>:25(use_fast)
<ipython-input-5-31b8b4df29c5>:25(use_fast)  <-       1    0.001    0.111  <string>:1(<module>)




In [22]:
stats.print_callees('use_fast')

   Ordered by: internal time
   List reduced from 6 to 1 due to restriction <'use_fast'>

Function                                     called...
                                                 ncalls  tottime  cumtime
<ipython-input-5-31b8b4df29c5>:25(use_fast)  ->     100    0.001    0.111  <ipython-input-5-31b8b4df29c5>:13(fast)




In [23]:
#notebook specific...
%prun use_fast()

In [24]:
%prun?

In [25]:
%prun -s time -q -T fast.txt use_fast()

 
*** Profile printout saved to text file 'fast.txt'. 


In [26]:
# %load clock_check.py
# file: measuring/clock_check.py

"""Checking different timing functions.
"""

from __future__ import print_function

import os
import sys
import time
import timeit


if sys.version_info.major < 3:
    range = xrange


def clock_check(duration=1):
    """Check the measured time with different methods.
    """
    start_os_time0 = os.times()[0]
    start_time_clock = time.clock()
    start_default_timer = timeit.default_timer()
    for _ in range(int(1e6)):
        1 + 1
    time.sleep(duration)
    durtation_os_time0 = os.times()[0] - start_os_time0
    durtation_time_clock = time.clock() - start_time_clock
    durtation_default_timer = timeit.default_timer() - start_default_timer
    print('durtation_os_time0:     ', durtation_os_time0)
    print('durtation_time_clock:   ', durtation_time_clock)
    print('durtation_default_timer:', durtation_default_timer)


if __name__ == '__main__':
    clock_check()


durtation_os_time0:      0.11999999999999988
durtation_time_clock:    0.12174700000000005
durtation_default_timer: 1.128159231004247


prefered method of timing is timeit - it is abstracted for windows and linux, py2 and py3. 

In [27]:
timeit.default_timer

<function time.perf_counter>

In [28]:
# %load cpu_time.py
# file: cpu_time.py

"""Measuring CPU time instead of wall clock time.
"""

import cProfile
import os
import sys
import time

# Make it work with Python 2 and Python 3.
if sys.version_info.major < 3:
    range = xrange


def cpu_time():
    """Function for cpu time. Os dependent.
    """
    if sys.platform == 'win32':
        return os.times()[0]
    else:
        return time.clock()


def sleep():
    """Wait 2 seconds.
    """
    time.sleep(2)


def count():
    """100 million loops.
    """
    for _ in range(int(1e8)):
        1 + 1


def test():
    """Run functions
    """
    sleep()
    count()


def profile():
    """Profile with wall clock and cpu time.
    """
    profiler = cProfile.Profile()
    profiler.run('test()')
    profiler.print_stats()

    profiler = cProfile.Profile(cpu_time)
    profiler.run('test()')
    profiler.print_stats()

if __name__ == '__main__':
    profile()


         7 function calls in 8.229 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.002    2.002 <ipython-input-28-d2afec7be092>:26(sleep)
        1    6.227    6.227    6.227    6.227 <ipython-input-28-d2afec7be092>:32(count)
        1    0.000    0.000    8.229    8.229 <ipython-input-28-d2afec7be092>:39(test)
        1    0.000    0.000    8.229    8.229 <string>:1(<module>)
        1    0.000    0.000    8.229    8.229 {built-in method exec}
        1    2.002    2.002    2.002    2.002 {built-in method sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         7 function calls in 6.014 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.003    0.003 <ipython-input-28-d2afec7be092>:26(sleep)
        1    6.011    6.011    6.011    6.011 <ipython

In [29]:
%timeit 1 + 1

30 ns ± 0.257 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [30]:
import time

In [31]:
%%timeit -n 1 -r 1

for _ in range(10**3):
    time.sleep(1e-3)

1.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


%timeit? was used to find what is going wrong by limiting the runs and loops - we found that we were using something too coarse for time.sleep

In [32]:
cd ../pi/

/home/mark/Projects/jupyter/pycon2017t1/pi


In [33]:
# %load simple_pi.py
# file: simple_pi.py

"""Calculating pi with Monte Carlo.
"""

from __future__ import print_function

import math
import random
import sys


if sys.version_info[0] < 3:
    range = xrange


def pi_plain(total):
    """Calculate pi with `total` hits.
    """
    count_inside = 0
    for _ in range(total):
        x = random.random()
        y = random.random()
        dist = math.sqrt(x * x + y * y)
        if dist < 1:
            count_inside += 1
    return 4.0 * count_inside / total

if __name__ == '__main__':

    def test():
        """Check if it works.
        """
        n = int(1e6)
        print('pi:', pi_plain(n))

    test()


pi: 3.142408


In [34]:
# %load numpy_pi.py
# file: numpy_pi.py
"""Calculating pi with Monte Carlo Method and NumPy.
"""

from __future__ import print_function

import numpy                                                   #1


def pi_numpy(total):                                           #2
    """Compute pi.
    """
    x = numpy.random.rand(total)                               #3
    y = numpy.random.rand(total)                               #4
    dist = numpy.sqrt(x * x + y * y)                           #5
    count_inside = len(dist[dist < 1])                         #6
    return 4.0 * count_inside / total

if __name__ == '__main__':

    def test():
        """Time the execution.
        """
        import timeit
        start = timeit.default_timer()
        pi_numpy(int(1e6))
        print('run time', timeit.default_timer() - start)
    test()


run time 0.11167853899678448


In [35]:
n = int(1e6)

In [36]:
%timeit pi_plain(n)

1.1 s ± 6.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%timeit pi_numpy(n)

109 ms ± 424 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


python -m cProfile -o pi.stat simple_pi.py

snakeviz pi.stat

In [38]:
%load_ext snakeviz

ImportError: No module named 'snakeviz'

In [66]:
%snakeviz pi_numpy(n)

ERROR:root:Line magic function `%snakeviz` not found.


In [67]:
cd ../measuring/

/home/mark/Projects/jupyter/pycon2017t1/measuring


In [68]:
ls

accumulate.py        local_ref.py              profile_me_use_line_profiler.py
calc.py              make_list_gen.py          profile_speed.py
clock_check.py       memory_growth_hpy.py      pympler_list_growth.py
cpu_time.py          memory_growth_pympler.py  pystone2_3.py
create_list.py       memory_size_hpy.py        pystone2.py
fast.stats           memory_size_pympler.py    pystone3.py
fast.txt             profile_me.py             pystone_converter.py
list_alloc_steps.py  profile_me.py.prof        use_mem.py


In [ ]:
# %load profile_me_use_line_profiler.py
# file profile_me_use_line_profiler.py

"""Example to be profiled.
"""

import time
import sys

if sys.version_info.major < 3:
    range = xrange


def fast():
    """Wait 0.001 seconds.
    """
    time.sleep(1e-3)


def slow():
    """Wait 0.1 seconds.
    """
    time.sleep(0.1)

@profile
def use_fast():
    """Call `fast` 100 times.
    """
    for _ in range(100):
        fast()

@profile
def use_slow():
    """Call `slow` 100 times.
    """
    for _ in range(100):
        slow()


if __name__ == '__main__':
    use_fast()
    use_slow()


In [71]:
# %load accumulate.py
# file accumulate.py

"""Simple test function for line_profiler.
"""

@profile
def accumulate(iterable):
    """Accumulate the intermediate steps in summing all elements.

    The result is a list with the length of `iterable`.
    The last element is the sum of all elements of `iterable`
    >>> accumulate(range(5))
    [0, 1, 3, 6, 10]
    >>> accumulate(range(10))
    [0, 1, 3, 6, 10, 15, 21, 28, 36, 45]
    """
    acm = [iterable[0]]
    for elem in iterable[1:]:
        old_value = acm[-1]
        new_value = old_value + elem
        acm.append(new_value)
    return acm


if __name__ == '__main__':
    accumulate(range(10))
    accumulate(range(100))


TypeError: profile() takes 0 positional arguments but 1 was given

In [ ]:
# %load calc.py
#calc.py

"""Simple test function for line_profiler doing some math.
"""

import math
import sys

if sys.version_info.major < 3:
    range = xrange


@profile
def calc(number, loops=1000):
    """Do some math calculations.
    """
    sqrt = math.sqrt
    for x in range(loops):
        x = number + 10
        x = number * 10
        x = number ** 10
        x = pow(number, 10)
        x = math.sqrt(number)
        x = sqrt(number)
        math.sqrt
        sqrt

if __name__ == '__main__':
    calc(100, int(1e5))


In [3]:
import sys
sys.executable

'/usr/bin/python3'

In [74]:
%load_ext line_profiler

ImportError: No module named 'line_profiler'

In [75]:
%lprun -f use_fase -f fast use_fast()

ERROR:root:Line magic function `%lprun` not found.


In [ ]:
# %load create_list.py
# file: create_list.py

import sys

if sys.version_info.major < 3:
    range = xrange


def insert_zero(n=int(1e4)):
    """Assemble list with `insert`. Inefficient.
    """
    L = []
    for x in range(n):
        L.insert(0, x)
    return L


def append_reverse(n=int(1e4)):
    """Assemble list with `append` and `reverse`.
    """
    L = []
    for x in range(n):
        L.append(x)
    L.reverse()
    return L
